# Scoring your trained model

In the cell below, please load your model into `model`. Also if you used an image size for your input images that *isn't* 224x224, you'll need to set `image_size` to the size you used. The scoring code assumes square input images.

For example, this is how I loaded in my checkpoint:

```python
import torch
from torch import nn
import torch.nn.functional as F
from torchvision import models

class FFClassifier(nn.Module):
    
    def __init__(self, in_features, hidden_features, 
                       out_features, drop_prob=0.1):
        super().__init__()
        
        self.fc1 = nn.Linear(in_features, hidden_features)
        self.fc2 = nn.Linear(hidden_features, out_features)
        self.drop = nn.Dropout(p=drop_prob)
        
    def forward(self, x):
        x = self.drop(F.relu(self.fc1(x)))
        x = self.fc2(x)
        x = F.log_softmax(x, dim=1)
        return x

    
def load_checkpoint(checkpoint_path):
    checkpoint = torch.load(checkpoint_path)
    
    model = models.vgg16(pretrained=False)
    for param in model.parameters():
        param.requires_grad = False

    # Put the classifier on the pretrained network
    model.classifier = FFClassifier(25088, checkpoint['hidden'], 102)
    
    model.load_state_dict(checkpoint['state_dict'])
    
    return model

model = load_checkpoint('/home/workspace/classifier.pt')
```

Your exact code here will depend on how you defined your network in the project. Make sure you use the absolute path to your checkpoint which should have been uploaded to the `/home/workspace` directory.

Run the cell, then after loading the data, press "Test Code" below. This can take a few minutes or more depending on the size of your network. Your model needs  to reach **at least 20% accuracy** on the test set to be recorded.

In [ ]:
!pip install torch

    35% |███████████▍                    | 211.5MB 24.1MB/s eta 0:00:16

    69% |██████████████████████▍         | 414.1MB 17.2MB/s eta 0:00:11    36% |███████████▋                    | 215.1MB 23.6MB/s eta 0:00:16    36% |███████████▊                    | 216.3MB 33.7MB/s eta 0:00:12    36% |███████████▊                    | 217.6MB 28.7MB/s eta 0:00:14    37% |███████████▉                    | 219.0MB 32.7MB/s eta 0:00:12    37% |████████████                    | 220.3MB 24.3MB/s eta 0:00:16    37% |████████████                    | 221.6MB 27.1MB/s eta 0:00:14    37% |████████████                    | 222.9MB 51.9MB/s eta 0:00:08    37% |████████████▏                   | 224.2MB 19.2MB/s eta 0:00:20    38% |████████████▏                   | 225.3MB 27.3MB/s eta 0:00:14    38% |████████████▎                   | 226.6MB 45.6MB/s eta 0:00:09    38% |████████████▍                   | 228.1MB 26.9MB/s eta 0:00:14    38% |████████████▍                   | 229.4MB 24.9MB/s eta 0:00:15MB/s eta 0:00:18    39% |████████████▋                   | 233.4MB 31.8MB/s e

    70% |███████████████�

    70% |███�

    70% |██████████████████████▋         | 417.7MB 13.4MB/s

...

    71% |██████████�

    71% |█

    71% |███████████████████████         | 424.6MB 

    72% |█████████████████████�

    99% |████████████████████████████████| 591.7MB 22.2MB/s eta 0:00:01�█▏        | 428.6MB 22.8MB/s eta 0:00:08��████████▎        | 429.8MB 15.5MB/s eta 0:00:11.3MB 20.3MB/s eta 0:00:08��████████████▍        | 432.3MB 12.6MB/s eta 0:00:13  | 433.8MB 51.5MB/s eta 0:00:04�████▌        | 434.9MB 18.3MB/s eta 0:00:09��████████████████████▋        | 436.6MB 57.1MB/s eta 0:00:03��███████████▊        | 437.9MB 15.7MB/s eta 0:00:100:00:0324.9MB/s eta 0:00:07�▉        | 441.2MB 29.0MB/s eta 0:00:06�██████████████        | 443.0MB 26.4MB/s eta 0:00:06��█████████████████████        | 444.1MB 18.7MB/s eta 0:00:08�██████████████████        | 445.5MB 31.3MB/s eta 0:00:05/s eta 0:00:10/s eta 0:00:05/s eta 0:00:07/s eta 0:00:06/s eta 0:00:03% |████████████████████████▋       | 455.4MB 20.1MB/s eta 0:00:07% |████████████████████████▊       | 456.6MB 21.5MB/s eta 0:00:07% |████████████████████████▊       | 458.1MB 20.3MB/s eta 0:00:07% |████████████████████████▉       | 459.2MB 14.7MB/s eta 0:00:09% |█

In [ ]:
import torch
from torch import optim
from torch.optim import lr_scheduler
from torchvision import models
from collections import namedtuple
from torch import nn
import torch.nn.functional as F
from torch.autograd import Variable
from torchvision import datasets, models, transforms
from torch.utils.data.sampler import SubsetRandomSampler
import time
import json
import copy
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import PIL
from PIL import Image
from collections import OrderedDict

In [ ]:
def load_checkpoint(filepath):
    checkpoint = torch.load(filepath, map_location="cpu")
    model.load_state_dict(checkpoint['state_dict'], strict=False)
    return model

model = models.densenet201(pretrained=True)

for param in model.parameters():
    param.requires_grad = False
    
    classifier = nn.Sequential(OrderedDict([
                          ('fc1', nn.Linear(1920, 4096)),
                          ('relu', nn.ReLU()),
                          ('fc2', nn.Linear(4096, 102)),
                          ('output', nn.LogSoftmax(dim=1))
                          ]))
model.classifier = classifier

In [ ]:
# Load your model to this variable
model = load_checkpoint('/home/workspace/classifier_densenet201_colab.pth')
   
# If you used something other than 224x224 cropped images, set the correct size here
image_size = 224
# Values you used for normalizing the images. Default here are for 
# pretrained models from torchvision.
norm_mean = [0.485, 0.456, 0.406]
norm_std = [0.229, 0.224, 0.225]